<a href="https://colab.research.google.com/github/mizoru/pronunciation-trainer/blob/main/training_thunder_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install thunder-speech

In [ ]:
!git clone https://github.com/mizoru/pronunciation-trainer.git

In [5]:
from kaggle import api
api.dataset_download_files('mfekadu/darpa-timit-acousticphonetic-continuous-speech')
import zipfile
archive = zipfile.ZipFile('darpa-timit-acousticphonetic-continuous-speech.zip')
archive.extractall()


In [6]:
import thunder
from thunder.callbacks import FinetuneEncoderDecoder

In [7]:
thunder.__version__

'3.1.1'

In [8]:
from thunder.data.dataset import BaseSpeechDataset
from thunder.data.datamodule import BaseDataModule
from typing import Any, List, Sequence, Tuple, Union
from torch import Tensor, nn
from thunder.registry import load_pretrained
from thunder.quartznet.compatibility import QuartznetCheckpoint
from pathlib import Path

In [9]:
import pandas as pd

In [10]:
labels = pd.read_csv('pronunciation-trainer/dataDS.csv')

In [11]:
def replace_slashes(s:str):
    return s.replace('\\', '/')

In [12]:
def gggparent_eq(p, gggp = "TEST"):
    return Path(p).parent.parent.parent.name == gggp

In [13]:
labels.Path = labels.Path.map(replace_slashes)

In [14]:
labels["is_valid"] = labels.Path.map(gggparent_eq)

In [15]:
class TimitDataset(BaseSpeechDataset):
  def open_audio(self, item) -> Tuple[Tensor, int]:
    return self.loader.open_audio(item.Path)
  def open_text(self, item) -> str:
    return item.Transcription
  def get_item(self, index: int) -> Any:
    return self.items.iloc[index]

In [ ]:
BaseDataModule?

In [16]:
class TimtiDataModule(BaseDataModule):
  def get_dataset(self, split):
    if split != "train":
      return TimitDataset(labels[labels["is_valid"]])
    else:
      return TimitDataset(labels[labels["is_valid"] == False])

In [16]:
ds = TimitDataset(labels)

In [17]:
ds.get_item(0)

Unnamed: 0                                                       0
Path                             ./data/TEST/DR1/FAKS0/SA1.WAV.wav
Transcription    ʃ.i.ɦ.æ.d̚.d.j.ɝ.d̚.d.ɑ.r.k̚.k.s.u.ɾ.ɪ.ŋ.ɡ̚.g....
is_valid                                                      True
Name: 0, dtype: object

In [18]:
dm = TimtiDataModule()
dm.setup()

In [19]:
def remove_dot(s):
  return ''.join(s.split('.'))

In [20]:
labels.Transcription = labels.Transcription.map(remove_dot)

In [21]:
whole = ''.join([t for t in labels.Transcription])

In [22]:
tokens = list(set(whole))

In [23]:
len(tokens)

48

In [4]:
from thunder.finetune import FinetuneCTCModule
from thunder.blocks import conv1d_decoder
import pytorch_lightning as pl

In [25]:
BaseDataModule?

In [26]:
model = FinetuneCTCModule(QuartznetCheckpoint.QuartzNet15x5Base_En, decoder_class = conv1d_decoder, tokens = tokens)

In [27]:
trainer = pl.Trainer(
    gpus=-1, # Use all gpus
    max_epochs=10,
    callbacks=[FinetuneEncoderDecoder(unfreeze_encoder_at_epoch=1)],
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [29]:
trainer.fit?

In [35]:
trainer.fit(model = model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type                 | Params | In sizes              | Out sizes             
----------------------------------------------------------------------------------------------------------
0 | encoder         | MultiSequential      | 18.9 M | [[10, 64, 101], [10]] | [[10, 1024, 51], [10]]
1 | decoder         | Conv1d               | 50.2 K | [10, 1024, 51]        | [10, 49, 51]          
2 | audio_transform | MultiSequential      | 0      | [[10, 16000], [10]]   | [[10, 64, 101], [10]] 
3 | text_transform  | BatchTextTransformer | 0      | ?                     | ?                     
4 | validation_cer  | CharErrorRate        | 0      | ?                     | ?                     
5 | validation_wer  | WordErrorRate        | 0      | ?                     | ?                     
----------------------------------------------------------------------------------------------------------
127 K     Trainable params
18.8 M   

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/data.py:73: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 10. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  "Trying to infer the `batch_size` from an ambiguous collection. The batch size we"


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [37]:
trainer.validate(model = model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      loss/val_loss         0.7184278964996338
       metrics/cer          0.22450698912143707
       metrics/wer          0.9916666746139526
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss/val_loss': 0.7184278964996338,
  'metrics/cer': 0.22450698912143707,
  'metrics/wer': 0.9916666746139526}]

In [1]:
from thunder.data.dataset import AudioFileLoader

In [2]:
loader = AudioFileLoader(sample_rate=16000)

In [7]:
from IPython.display import Audio

In [50]:
Audio(labels.Path[8])

In [53]:
Path.cwd()

PosixPath('/content')

In [52]:
model.predict(loader(labels.Path[8]))

['ɝwɔrd̚drɛlp̚pkɨ̞ntɛssəvoʊnlisk̚kɑt̚tɨ̞nb̚blaɪvfsɨ̞z']

In [ ]:
model.to_torchscript("QuartzNet_thunderspeech_1.pt")

In [40]:
model.to_torchscript?

In [61]:
!cd pronunciation-trainer !ls

/bin/bash: line 0: cd: too many arguments


In [59]:
!ls

darpa-timit-acousticphonetic-continuous-speech.zip  SPKRINFO.TXT
data						    SPKRSENT.TXT
lightning_logs					    test_data.csv
PHONCODE.DOC					    TESTSET.DOC
PROMPTS.TXT					    TIMITDIC.DOC
pronunciation-trainer				    TIMITDIC.TXT
README.DOC					    train_data.csv
sample_data


In [62]:
!git add pronunciation-trainer/.

fatal: not a git repository (or any of the parent directories): .git


In [13]:
import torch
import pandas as pd
from pathlib import Path

In [4]:
model = torch.jit.load("QuartzNet_thunderspeech_1.pt")

In [8]:
natives = pd.read_csv('pronunciation-trainer/natives.csv')

In [9]:
subset = ["thin", "thing", "think", "fit", "feet", "bald", "bold", "food", "foot", "death", "deaf", "worm", "warm"]

In [10]:
subset_df = natives[natives.replica.isin(subset)]

In [20]:
!wget https://lingualibre.org/datasets/Q22-eng-English.zip

--2022-05-10 20:11:32--  https://lingualibre.org/datasets/Q22-eng-English.zip
Resolving lingualibre.org (lingualibre.org)... 152.228.161.167, 2001:41d0:304:100::4790
Connecting to lingualibre.org (lingualibre.org)|152.228.161.167|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 305389673 (291M) [application/zip]
Saving to: ‘Q22-eng-English.zip’

Q22-eng-English.zip 100%[===================>] 291.24M  11.6MB/s    in 25s     

2022-05-10 20:11:59 (11.4 MB/s) - ‘Q22-eng-English.zip’ saved [305389673/305389673]



In [21]:
import zipfile
archive = zipfile.ZipFile('Q22-eng-English.zip')
archive.extractall()

In [ ]:
!sudo apt install sox

In [53]:
!ls

In [47]:
!sox "Q22-eng-English/natives/Back ache/food.ogg" -r 16000 -c 1 -b 16 "Q22-eng-English/natives/Back ache/food.wav"

sox FAIL formats: can't open input file `Q22-eng-English/natives/Back ache/food.ogg': No such file or directory


In [11]:
subset_df

,Unnamed: 0,artist,replica,path
3410,3451,Back ache,death,./natives/Back ache/death.ogg
3623,3667,Back ache,feet,./natives/Back ache/feet.ogg
3670,3714,Back ache,food,./natives/Back ache/food.ogg
3671,3715,Back ache,foot,./natives/Back ache/foot.ogg
4754,4818,Back ache,thin,./natives/Back ache/thin.ogg
4755,4819,Back ache,thing,./natives/Back ache/thing.ogg
4757,4821,Back ache,think,./natives/Back ache/think.ogg
4917,5038,Back ache,warm,./natives/Back ache/warm.ogg
5002,5124,Back ache,worm,./natives/Back ache/worm.ogg
5355,5534,Exilexi,thin,./natives/Exilexi/thin.ogg


In [14]:
(Path('Q22-eng-English') / '/'.join(subset_df.path.iloc[2].split('/')[2:])).exists()

True

In [74]:
(Path('Q22-eng-English') / '/'.join(subset_df.path.iloc[2].split('/')[2:]))#.is_file()

PosixPath('Q22-eng-English/Back ache/food.ogg')

In [ ]:
subset_df["transcription"] = subset_df.path.map(lambda x: 
model.predict(loader(Path('Q22-eng-English') / '/'.join(x.split('/')[2:]))))

In [18]:
?Audio

In [15]:
Audio(str(Path('Q22-eng-English') / '/'.join(subset_df.path.iloc[1].split('/')[2:])))

In [28]:
len(loader(str(Path('Q22-eng-English') / '/'.join(subset_df.path.iloc[1].split('/')[2:])))[0])

14123

In [22]:
model.predict?

In [ ]:
loader()

In [ ]:
predicts = []
for i in range(len(subset_df)):
  path = str(Path('Q22-eng-English') / '/'.join(subset_df.path.iloc[i].split('/')[2:]))
  print(path)
  audio = loader(path)
  predicts.append(model.predict(audio))
  print(predicts[-1])

Q22-eng-English/Back ache/death.ogg
['dɛf']
Q22-eng-English/Back ache/feet.ogg


In [60]:
model.predict(loader(Path('Q22-eng-English') / '/'.join([subset_df.path.iloc[2].split('/')][1:])))

RuntimeError: ignored

In [2]:
(Path('Q22-eng-English') / subset_df.path.iloc[2]).is_file()

NameError: ignored

In [57]:
Path('Q22-eng-English') / subset_df.path.iloc[2]

PosixPath('Q22-eng-English/natives/Back ache/food.ogg')

In [95]:
Path.PATH

AttributeError: ignored

In [82]:
Path('content/Q22-eng-English/Back ache/&.ogg').is_file()

False